-sandbox
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

#Convolutional Neural Networks

We will use pre-trained Convolutional Neural Networks (CNNs), trained with the image dataset from [ImageNet](http://www.image-net.org/), to demonstrate two aspects. First, how to explore and classify images. And second, how to use transfer learning with existing trained models (next lab).

## VGG16
![vgg16](https://brookewenig.github.io/img/DL/vgg16.png)

We are going to start with the VGG16 model, which was introduced by Simonyan and Zisserman in their 2014 paper [Very Deep Convolutional Networks for Large Scale Image Recognition](https://arxiv.org/abs/1409.1556).

Let's start by downloading VGG's weights and model architecture.

In [4]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions, VGG16
import numpy as np
import os

vgg16Model = VGG16(weights='imagenet')

We can look at the model summary. Look at how many parameters there are! Imagine if you had to train all 138,357,544 parameters from scratch! This is one motivation for re-using existing model weights.

**RECAP**: What is a convolution? Max pooling?

In [6]:
vgg16Model.summary()

**Question**: What do the input and output shapes represent?

In Tensorflow, it represents the images in a channels-last manner: (samples, height, width, color_depth)

But in other frameworks, such as Theano, the same data would be represented channels-first: (samples, color_depth, height, width)

## Apply pre-trained model

We are going to make a helper method to resize our images to be 224 x 224, and output the top 3 classes for a given image.

In [10]:
def predict_images(images, model):
  for i in images:
    print ('processing image:', i)
    img = image.load_img(i, target_size=(224, 224))
    #convert to numpy array for Keras image formate processing
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    # decode the results into a list of tuples (class, description, probability
    print('Predicted:', decode_predictions(preds, top=3)[0], '\n')

-sandbox
## Images
<div style="text-align: left; line-height: 0; padding-top: 9px;">
  <img src="https://brookewenig.github.io/img/DL/pug.jpg" height="150" width="150" alt="Databricks Nerds!" style=>
  <img src="https://brookewenig.github.io/img/DL/strawberries.jpg" height="150" width="150" alt="Databricks Nerds!" style=>
  <img src="https://brookewenig.github.io/img/DL/rose.jpg" height="150" width="150" alt="Databricks Nerds!" style=>
  
</div>

Let's make sure the datasets are already mounted.

In [12]:
%run "./Includes/Classroom Setup"

In [13]:
img_paths = ["/dbfs/mnt/training/dl/img/pug.jpg", "/dbfs/mnt/training/dl/img/strawberries.jpg", "/dbfs/mnt/training/dl/img/rose.jpg"]
predict_images(img_paths, vgg16Model)

The network did so well with the pug and strawberry! What happened with the rose? Well, it turns out that `rose` was not one of the 1000 categories that VGG16 had to predict. But it is quite interesting it predicted `sea_anemone` and `vase`.

You can play around with this with your own images by doing the following:

Get a new file: 

`%sh wget image_url.jpg`

`%fs cp file:/databricks/driver/image_name.jpg yourName/tmp/image_name.jpg `


OR

You can upload this file via the Data UI and read in from the FileStore path (e.g. `/dbfs/FileStore/image_name.jpg`).

# DeepImagePredictor

While it's great and fun to play around with Keras on the driver, what about loading a copy of the model to our workers, and make these predictions in parallel on our Spark cluster using [Deep Learning Pipelines](https://github.com/databricks/spark-deep-learning)! 

Deep Learning Pipelines is an open-source project started by Databricks for distributed model inference of deep learning models.

**NOTE**: We are running it on a small dataset, and there is quite high overhead of copying the model to our workers. This is useful when your datasets are quite large

-sandbox
## Classify Co-Founders of Databricks
<div style="text-align: left; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2017/12/Ali-Ghodsi-4.jpg" height="150" width="150" alt="Databricks Nerds!" style=>
  <img src="https://databricks.com/wp-content/uploads/2017/12/andy-konwinski-1.jpg" height="150" width="150" alt="Databricks Nerds!" style=>
  <img src="https://databricks.com/wp-content/uploads/2015/08/ionS.jpg" height="150" width="150" alt="Databricks Nerds!" style=>
  <img src="https://databricks.com/wp-content/uploads/2016/02/MateiZ.jpg" height="200" width="150" alt="Databricks Nerds!" style=>
  <img src="https://databricks.com/wp-content/uploads/2015/08/patrickW.jpg" height="150" width="150" alt="Databricks Nerds!" style=>
  <img src="https://databricks.com/wp-content/uploads/2017/12/Reynold-Xin.jpg" height="150" width="150" alt="Databricks Nerds!" style=>
</div>

In [18]:
from pyspark.ml.image import ImageSchema
from sparkdl.image import imageIO
from sparkdl import DeepImagePredictor

df = ImageSchema.readImages("mnt/training/dl/img/founders/")

predictor = DeepImagePredictor(inputCol="image", outputCol="predicted_labels", modelName="VGG16", decodePredictions=True, topK=5)
predictions_df = predictor.transform(df).cache()
display(predictions_df)

## Inception V3

What happened to Matei's predictions: Chain mail?? Well, he truly is Spark's knight in shining armor!

Let's change the model to a more recent architecture, and see what the predictions are now!

In [20]:
from pyspark.ml.image import ImageSchema
from sparkdl.image import imageIO
from sparkdl import DeepImagePredictor

df = ImageSchema.readImages("mnt/training/dl/img/founders/")

predictor = DeepImagePredictor(inputCol="image", outputCol="predicted_labels", modelName="InceptionV3", decodePredictions=True, topK=5)
predictions_df = predictor.transform(df).cache()
display(predictions_df)

Yikes! These are not the most natural predictions (because ImageNet did not have a `person` category). In the next lab, we will cover how to utilize existing components of the VGG16 architecture, and how to retrain the final classifier.

-sandbox
&copy; 2018 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>